# One-way route

## Imports

In [1]:
import googlemaps
import gmaps
import collections
from api_keys.api_key import api_key
collections.Iterable = collections.abc.Iterable
collections.Sequence = collections.abc.Sequence

## Instaciate the API

In [2]:
key = api_key
client = googlemaps.Client(key)
gmaps.configure(api_key=key)

## Users input

In [3]:
# user input
start = input('Enter the start point: ')
destination = input('Enter the final destination: ')
max_dist = float(input('Enter the duration to drive per day, in hours: ')) * 80 * 1000

Enter the start point: la coruna
Enter the final destination: barcelona
Enter the duration to drive per day, in hours: 3


## Defining the coordinates of the origin and the destination

### Origin

In [4]:
def origin_coord(start):
    start_id = client.places_autocomplete(start)[0]['place_id']

    start_coord = client.geocode(place_id=start_id)[0]['geometry']['location']

    start_list = (start_coord['lat'], start_coord['lng'])
    
    return start_list

In [5]:
origin_location = origin_coord(start)
origin_location

(43.3623436, -8.4115401)

### Destination

In [6]:
def destination_coord(destination):
    destination_id = client.places_autocomplete(destination)[0]['place_id']

    destination_coord = client.geocode(place_id=destination_id)[0]['geometry']['location']

    dest_list = (destination_coord['lat'], destination_coord['lng'])
    return dest_list

In [7]:
destination_location = destination_coord(destination)
destination_location

(41.3873974, 2.168568)

## Split the route based on the maximum distance to be traveled per day

In [8]:
def split_route(start, destination, max_dist):
# Takes the origin and the destination and split the route into smaller pieces and get
# the coordinates of every point of the route.
    origin = client.places_autocomplete(start)[0]['description']
    destination = client.places_autocomplete(destination)[0]['description']

    route = client.directions(origin, destination, mode='driving')[0]['legs'][0]['steps']

    sum = 0
    coordinates = []
    for step in route:
        sum = sum + step['distance']['value']
        if sum >= max_dist:
            sum = 0
            coordinates.append(step['start_location'])
            
    coordinates.append(client.directions(origin, destination, mode='driving')[0]['legs'][0]['steps'][-1]['end_location'])
    
    stopover = []
    
    for coord in coordinates:
        coord_tuple = (coord['lat'], coord['lng'])
        stopover.append(coord_tuple)
    
    return stopover

In [9]:
route_coord = split_route(start, destination, max_dist)

### Defining the waypoints coordinates

In [10]:
def waypoints(stop_coord):
    waypoints = stop_coord[1:-1]
    
    city_waypoint = []
    for coord in waypoints:
        city_location = client.places_nearby(coord, radius=20000, type='locality')['results'][0]['geometry']['location']
        city_coord = (city_location['lat'], city_location['lng'])
        city_waypoint.append(city_coord)
    
    return city_waypoint

In [11]:
waypoint_coord = waypoints(route_coord)
waypoint_coord

[(42.3504659, -3.689354499999999),
 (41.6488226, -0.8890853),
 (41.2196699, 1.534307)]

## Finding nearest hotels and restaurant based in each stopover

In [14]:
def hotel_marker_single(place_coord):
    hotel_dest = client.places(location=place_coord, radius=100000, type='landmark')['results']

    hotel_names = []
#     hotel_rating = []
    hotel_coord = []
    hotel_adress = []

    for result in hotel_dest:
        dictionary1 = {'Name': result['name']}
        hotel_names.append(dictionary1)

#         dictionary2 = {'Rating': result['rating']}
#         hotel_rating.append(dictionary2)

        tuple1 = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
        dictionary3 = {'location': tuple1}
        hotel_coord.append(dictionary3)
        
        dictionary4 = {'Adress': result['vicinity']}
        hotel_adress.append(dictionary4)
        
    hotel_info = []
    
    for i in range(len(hotel_names)):
        temp_dict = {}
        temp_dict.update(hotel_names[i])
        temp_dict.update(hotel_coord[i])
#         temp_dict.update(hotel_rating[i])
        temp_dict.update(hotel_adress[i])
        hotel_info.append(temp_dict)
        
    return hotel_info

In [140]:
def location_marker(waypoint_coord,type_location):
    location_dest=[]
    
    for stop in waypoint_coord:
        location_search =[]
        locations = client.places_nearby(stop, radius=100000, type=type_location)['results']
        location_search.append(locations)

        location_names = []
        location_rating = []
        location_coord = []
        location_adress = []
        
        for result in location_search[0]:
            dictionary1 = {'name': result['name']}
            location_names.append(dictionary1)
            try:
                dictionary2 = {'rating': result['rating']}
                location_rating.append(dictionary2)
            except:
                dictionary2 = {'rating': "No Rating"}
                location_rating.append(dictionary2)
            
            tuple1 = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
            dictionary3 = {'location': tuple1}
            location_coord.append(dictionary3)

            dictionary4 = {'Adress': result['vicinity']}
            location_adress.append(dictionary4)

        location_info = []

        for i in range(len(location_names)):
            temp_dict = {}
            temp_dict.update(location_names[i])
            temp_dict.update(location_coord[i])
            temp_dict.update(location_rating[i])
            temp_dict.update(location_adress[i])
        
            location_info.append(temp_dict)
        
        location_dest.append(location_info)
        all_locations = []
        for i in range(len(location_dest)):
            all_locations = all_locations + location_dest[i]
    return all_locations

In [139]:
hotel_dest = location_marker(waypoint_coord, 'lodging')
hotel_dest

[{'name': 'ibis Irun',
  'location': (43.332663, -1.816728),
  'rating': 4,
  'Adress': 'Letxumborro Hiribidea, 77, Hss0660, Irun'},
 {'name': 'Parador de Hondarribia',
  'location': (43.363551, -1.791278000000001),
  'rating': 4.4,
  'Adress': 'Plaza de Armas, 14, Hondarribia'},
 {'name': 'Hotel Jaizkibel',
  'location': (43.3663964, -1.798624999999999),
  'rating': 4.5,
  'Adress': 'Baserritar Etorbidea, 1, Hondarribia'},
 {'name': 'KYRIAD DIRECT ETH IRUN',
  'location': (43.33845089999999, -1.7542218),
  'rating': 3.9,
  'Adress': 'Calle Comete Sarea, 2, Irun'},
 {'name': 'Hotel Obispo',
  'location': (43.36265969999999, -1.7909471),
  'rating': 4.3,
  'Adress': 'Apezpiku Kalea, 1, Hondarribia'},
 {'name': 'Sercotel Jauregui',
  'location': (43.36804929999999, -1.7936897),
  'rating': 4.3,
  'Adress': 'Zuloaga Kalea, 5, Hondarribia'},
 {'name': 'Campanile Hendaye',
  'location': (43.34779199999999, -1.767688),
  'rating': 3.9,
  'Adress': '102 Rue de Béhobie, Hendaye'},
 {'name': "C

In [128]:
rest_dest = location_marker(waypoint_coord, 'restaurant')
rest_dest

[{'Name': 'Campanile Hendaye',
  'location': (43.34779199999999, -1.767688),
  'Rating': 3.9,
  'Adress': '102 Rue de Béhobie, Hendaye'},
 {'Name': 'Les Jardins de Bakea',
  'location': (43.33349150000001, -1.7444378),
  'Rating': 4.6,
  'Adress': '1134 Rue Herri Alde, Biriatou'},
 {'Name': 'Hotel Urdanibia Park',
  'location': (43.3298491, -1.8257869),
  'Rating': 4,
  'Adress': 'Jaizubia Hiribidea, 61, Irun'},
 {'Name': 'Hôtel Restaurant Bergeret Sport',
  'location': (43.3712698, -1.772369),
  'Rating': 4.7,
  'Adress': '4 Rue des Clématites, Hendaye'},
 {'Name': 'Auberge chez Maïté',
  'location': (43.36214440000001, -1.6999267),
  'Rating': 4.2,
  'Adress': '1 Rue Clément Laurencena, Place de la Mairie, Urrugne'},
 {'Name': 'Restaurant Arotzenia',
  'location': (43.3618235, -1.703702),
  'Rating': 4.5,
  'Adress': '47 Rue Jean Fourcade, Urrugne'},
 {'Name': 'Usategieta',
  'location': (43.32239329999999, -1.8518644),
  'Rating': 4.4,
  'Adress': 'Maldaburu Bidea, 15'},
 {'Name': '

## Generate the map, with the main route and the stopover, including hotels and restaurants for each stop.

In [134]:
def main_route(origin_coord, destination_coord, waypoints, hotel_markers, rest_markers):
    fig = gmaps.figure()
    
    main = gmaps.directions_layer(origin_coord, destination_coord, waypoints=waypoints)
    
    hotel_location = [hotel['location'] for hotel in hotel_markers]
    info_box_template = """
    <dl>
    <dt>Name</dt><dd>{Name}</dd>
    <dt>Rating</dt><dd>{Rating}</dd>
    <dt>Adress</dt><dd>{Adress}</dd>
    </dl>
    """
    info_hotel = [info_box_template.format(**hotel) for hotel in hotel_markers]
    
    hotel_markers = gmaps.symbol_layer(hotel_location, hover_text='Hotel', fill_color='red', stroke_color='green',  scale=4, info_box_content=info_hotel)
    
    rest_location = [rest['location'] for rest in rest_markers]
    info_box_template = """
    <dl>
    <dt>Name</dt><dd>{Name}</dd>
    <dt>Rating</dt><dd>{Rating}</dd>
    <dt>Adress</dt><dd>{Adress}</dd>
    </dl>
    """
    info_rest = [info_box_template.format(**hotel) for hotel in rest_markers]
    
    rest_markers = gmaps.symbol_layer(rest_location, hover_text='Restaurant', fill_color='red', stroke_color='blue',  scale=4, info_box_content=info_rest)
    
    fig.add_layer(main)
    fig.add_layer(hotel_markers)
    fig.add_layer(rest_markers)
    
    return fig

In [135]:
main_route(origin_location, destination_location, waypoint_coord, hotel_dest, rest_dest)

Figure(layout=FigureLayout(height='420px'))